In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import numpy as np
import pandas as pd

import pickle
import jieba
import h5py

SOS_token = 0
EOS_token = 1

# save data

# load data

In [2]:
import dataProcess as dp
h5py_file = h5py.File('../data/train_afterProcess.h5py','r')
pairs = h5py_file['pairs']

print(pairs[0][0].decode('utf-8'))
print(pairs[0][1].decode('gb2312'))

inputlang = dp.Lang('en')
outputlang = dp.Lang('zh')
inputlang.load('../data/en_train.pkl')
outputlang.load('../data/zh_train.pkl')

print(inputlang.name,inputlang.n_words)
print(outputlang.name,outputlang.n_words)
#h5py_file.close()

a pair of red crowned cranes have staked out their nesting territory
 一 对 丹 顶 鹤 正 监 视 着 它 们 的 筑 巢 领 地 
error: Name error------------------------------!
error: Name error------------------------------!
en 388091
zh 6754


In [3]:
print(pairs[90987][0].decode('utf-8'))

it ll be just like we planned .


In [6]:
pair = random.choice(pairs)
print(pair[0].decode('utf-8'))
print(pair[1].decode('gb2312'))

now if you ll just take a seat we can begin .
 好 ， 请 坐 吧 ， 这 样 我 们 就 可 以 开 始 了 。 


In [ ]:
import numpy as np
tt = ['我是中国人'.encode('gb2312'),'sdfeow dsf df dsf '.encode('utf-8')]
h5 = h5py.File('../data/test1.h5py','w')
h5.create_dataset('pairs',data=tt,dtype = 'S400')
h5.close()

#  data analysis

In [ ]:
count = inputlang.word2count
wordFreqLess10 = 0#保存词频小于某个阈值的词
wordFreqMore100 = 0
all_word_count=0
print('input english word count:' , inputlang.n_words)
for word in count:
    all_word_count += count[word]
    if count[word] <= 10:
        wordFreqLess10 +=count[word]
    if count[word] <= 10:
        wordFreqMore100 +=1
print('word count larger than 100:',inputlang.n_words - wordFreqMore100)
print('word count Less than 10:',wordFreqLess10)
print('rate of word count less than 10:', float(wordFreqLess10)/float(all_word_count))

## filter input language data

In [ ]:
testdict = {'a':23,'b':2,'c':4}
test2 = {1:'d',34:'f',22:'t'}
print(test2)
test2.pop(34)
print(test2)
a = testdict.copy()
print(a)
for word in testdict:
    if testdict[word] < 3:
        a.pop(word)
print(a)

### 匹配生僻字与非法字符

In [ ]:
import re

def is_rare_name(string):
    pattern = re.compile(u"[~!@#$%^&* ]")
    match = pattern.search(string)
    if match:
        return True
    try:
        string.encode("gb2312")
    except UnicodeEncodeError:
        return True
    return False

def normalizeChinese(s):
    try:
        s.encode("gb2312")
    except UnicodeEncodeError:
        return ' '
    s = re.sub(r"[~!@#$%^&* ]+",r' ', s)
    return s


In [ ]:
str1 = '我 是 @ 中 国 人 ， 我 # 不 才 囧 怪。'
#str1 = 'I an # a boy'
l = [normalizeChinese(s) for s in str1.split(' ')]
print(l)
tmp = ' '
for s in str1.split(' '):
    val = normalizeChinese(s)
    tmp += val+' '
print(tmp)

In [ ]:
result = re.search(r'.','I love FishC.com!')
print(result.group())

In [ ]:
s = '我 是 @ 中 国 人 ， 我 # 不 囧 才 怪。'
s = re.sub(r"[~!@#$%^&* ]+",r' ', s)
print (s)

# build models

In [ ]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

from hyperboard import Agent

agent = Agent(port=5100)
#agent = Agent(address='172.18.216.69',port=5000)
hyperparameters = {'test':0.1}
name = agent.register(hyperparameters, 'loss')

use_cuda = torch.cuda.is_available()
MAX_LENGTH = 30

# test filter data to model

In [ ]:
#返回词对应的下标
def indexesFromSentence(lang, sentence):
    #return [lang.word2index[word] for word in sentence.split(' ')]
    result = []
    all_lang_keys = lang.word2index.keys()
    for word in sentence.split(' '):
        if word in all_lang_keys:
            result.append(lang.word2index[word])
    return result

In [ ]:
test12 = 'a boy is watching tree.'
test11 = indexesFromSentence(inputlang, test12)
print(test11)


In [ ]:
testdict = {'a':1,'b':23,'c':4}
keys = testdict.keys()
list_result = []
for word in ['a','f','c','d']:
    if word in keys:
        list_result.append(testdict[word])
print(list_result)